In [2]:
import os
%pwd

'/Users/anshujoshi/PycharmProjects/CancerDetection/research'

In [3]:
os.chdir('..')
%pwd

'/Users/anshujoshi/PycharmProjects/CancerDetection'

In [4]:
from dataclasses import dataclass
from pathlib import Path
from typing import List


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_model_path: Path
    params_image_size: List
    params_include_top: bool
    params_learning_rate: float
    params_classes: int
    params_weight: str



In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

CONFIG_FILE_PATH set to: config/config.yaml
PARAMS_FILE_PATH set to: params.yaml


In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_model_path=Path(config.updated_base_model),
            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_learning_rate=self.params.LEARNINIG_RATE,
            params_classes=self.params.CLASSES,
            params_weight=self.params.WEIGHTS,
        )

        return prepare_base_model_config

In [7]:
import tensorflow as tf

In [8]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weight,
            include_top=self.config.params_include_top,
        )

        self.save_model(
            self.config.base_model_path,
            self.model
        )

    @staticmethod
    def _prepare_full_model(model: tf.keras.Model, classes: int, freeze_all: bool, freeze_till: int,
                            learning_rate: float) -> tf.keras.Model:
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif freeze_till is not None:
            for layer in model.layers[:freeze_till]:
                layer.trainable = False
            for layer in model.layers[freeze_till:]:
                layer.trainable = True
                
        x = model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        predictions = tf.keras.layers.Dense(classes, activation='softmax')(x)
        
        full_model = tf.keras.Model(inputs = model.input,outputs = predictions)
        
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy']  
        )
        
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate= self.config.params_learning_rate  
        )
        
        self.save_model(
            self.config.updated_model_path,
            self.full_model
        )
        

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path) 

In [9]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e
    

[2024-06-19 14:34:51,156: INFO: common: yaml config/config.yaml loaded successfully]
[2024-06-19 14:34:51,158: INFO: common: yaml params.yaml loaded successfully]
[2024-06-19 14:34:51,158: INFO: common: created directory at: artifacts]
[2024-06-19 14:34:51,159: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-06-19 14:34:51,454: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2024-06-19 14:34:51,590: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.]
[2024-06-19 14:34:51,592: WARNING: __init__: There is a known slowdown when using v2.11+ Keras optimizers on M1/M2 Macs. Falling back to the legacy Keras optimizer, i.e., `tf.keras.optimizers.legacy.Adam`.]


/Users/anshujoshi/anaconda3/envs/cancer/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
%pwd

'/Users/anshujoshi/PycharmProjects/CancerDetection/research'